并发：不同的代码块交替执行

并行：不同的代码块同时执行

# 秒杀系统

### 系统要求

高性能：秒杀涉及大量的并发读和并发写，因此要求性能必须高；

一致性：秒杀中商品减库存的实现方式同样关键。可想而知，有限数量的商品同一时刻被很多倍的请求同时来减库存。在大并发更新数据的过程中保证数据的准确性，难度可想而知；

高可用：出现问题的时候要保证可用；

# 架构原则

1. 数据要尽量少

   所谓数据尽量少，就是指用户请求的数据能少则少。请求的数据包括上传给系统的数据和系统返回给用户的数据（通常就是网页）。
   
   因为这些数据在网络上传输需要时间，其次不管是请求数据还是返回数据都需要服务器来处理，而服务器在写网络时都要做压缩和编码，这些都非常消耗CPU，所以减少传输的数据量可以显著的减少CPU的使用。例如，我们可以简化秒杀页面的大小，去掉不必要的页面装修效果等。
   
   其次，“数据要尽量少”还要求系统依赖的数据能少就少，包括系统完成某些业务逻辑需要读取和保存的数据，这些数据一般是和后台服务和数据库打交道的。调用其他服务会涉及到数据的序列化和反序列化，这也是CPU的一大杀手，同样也会增加延时。而且，数据库本身也容易成为一个瓶颈，所以和数据库打交道越少越好，越简单越好。
   

2. 请求数要尽量少

    用户请求的页面返回后，浏览器渲染页面还包含其他的额外请求，比如说这个页面依赖的CSS/javascript，图片以及Ajax请求等，这些额外的请求应该尽量少。应为浏览器没发出一个请求都多少会有一些消耗，例如建立连接要做三次握手，另外不同请求访问的域名不一样，还需要做DNS域名解析，可能会耗时更久。
    
    
3. 路径要尽量短

    所谓“路径”，就是用户发出请求到返回数据这个过程中，需求经过的中间的节点数。
    

4. 依赖要尽量少

    所谓依赖，指的是要完成一次用户请求必须依赖的系统或者服务，这里的依赖指的是强依赖。


5. 不要有单点
    
    系统中的单点可以说是系统架构上的一个大忌，因为单点意味着没有备份，风险不可控，我们设计分布式系统最重要的原则就是“消除单点”。
    

# 做好动静分离

### 何为动静数据

简单来说，“静态数据”和“动态数据”的主要区别就是看页面中输出的数据是否和URL、浏览器、时间、地域相关，以及是否含有Cookie等私密信息。

### 如何做动静分离的改造

下面我们从5个方面来分离动态内容：

1. URL唯一化：商品详情系统天然地就可以做到URL唯一化，比如每个商品都由ID来标识，那么http://item.xxx.com/item.htm?id=xxx 就可以作为唯一的URL标识。为啥要URL唯一呢？前面说了我们是要缓存整个HTTP连接，那么以什么作为Key呢？就以URL作为缓存的Key，例如id=xxx 这个格式进行区分。

2. 分离浏览者相关的因素：浏览者相关的因素包括是否已登录，以及登录身份等，这些因素可以单独拆出来，通过动态请求来获取。

3. 分离时间因素：服务器输出的时间也要动态请求来获取。

4. 异步化地域因素：详情页上与地域相关的因素做成异步获取。

5. 服务端输出的页面包含的Cookie可以通过代码软件来删除：如，web服务器Varnish可以通过

    `unset req.http.cookie`
    
    命令去掉Cookie。注意，这里说的去掉Cookie并不是用户端收到的页面不含Cookie了，而是说，在缓存的静态数据中不含有Cookie。

# 静态数据如何缓存

### 第一，你应该把静态数据缓存到离用户最近的地方。
静态数据就是那些相对不会变化的数据，因此我们可以把它们缓存起来。缓存到哪里呢？常见的有三种，用户浏览器里、CDN上或者在服务器端的Cache中。你因该根据情况，把它们尽量缓存到离用户最近的地方。
### 第二，静态化改造就是要直接缓存HTTP连接。
相较于普通的数据缓存而言，你肯定还听过系统发的静态化改造。静态化改造是直接缓存HTTP连接而不是仅仅缓存数据

例如：Web代理服务器根据请求URL直接取出对应的HTTP响应头和响应体然后直接返回，这个响应过程简单的连HTTP协议都不用重新组装，甚至连HTTP请求头也不需要解析。

# 动态数据如何处理

### 1. ESI方案（或者SSI）：
即在Web代理服务器上做动态内容请求，并将请求插入到静态页面中，当用户拿到页面时已经是一个完整的页面了。这种方式对服务器端性能有些影响，但用户体验较好。
### 2.CSI方案：
即单独发起一个异步JavaScript请求，以向服务端获取动态内容。这种方式服务器端性能更佳，但使用户端页面可能会延时，体验稍差。

# 如何处理热点数据

### 处理热点数据的通常几个思路：一是优化、二是限制、三是隔离。

先来说说优化。优化热点数据最有效的办法就是缓存热点数据，如果热点数据做了动静分离，那么可以长期缓存静态数据。但是，缓存热点数据更多的是“临时”缓存，即不管是静态数据还是动态数据，都用一个队列短暂地缓存数秒钟，由于队列长度有限，可以采用LRU淘汰算法替换。

再说说限制。限制更多的是一种保护机制，限制的办法也有很多，例如，对被访问商品的ID做一致性Hash，然后根据Hash做分桶，每个分桶设置一个处理队列，这样可以把热点商品限制在一个请求队列里，防止因某些热点商品占用太多的服务器资源，而是其他请求始终得不到服务器的处理资源。

最后介绍一下隔离。秒杀系统设计的第一个原则就是将这种热点数据隔离出来，不要让1%的请求影响到另外的99%，隔离出来后也更方便对这1%的请求做针对性的优化。

具体到“秒杀”业务，我们可以在一下几个层次实现隔离。
+ **业务隔离**: 把秒杀做成一种营销活动，卖家要参加秒杀这种营销活动需要单独报名，从技术上来说，卖家报名后对我们来说就有了已知热点，因此可以提前做好预热。
+ **系统隔离**: 系统隔离更多的是运用时的隔离，可以通过分组部署的方式和另外99%分开。秒杀可以申请单独的域名，目的也是让请求落到不同的集群中。
+ **数据隔离**: 秒杀所调用的数据大部分都是热点数据，比如会启用单独的Cache集群或者MySQL数据库来放热点数据，目的也是不想0.01%的数据有机会影响99.99%数据。

当然了，实现隔离有很多种方法。比如，你可以按照用户来区分，给不同的用户分配不用的Cookie，在接入层，路由到不同的服务器接口中；再比如，你还可以再接入层针对URL中的不同Path来设置限流策略。服务层调用不同的服务器接口，以及数据层通过给数据打标来区分等等这些措施，其目的都是把已经识别出来的热点请求和普通请求区分开。